In [4]:
#必要なパッケージをインポート
from requests_oauthlib import OAuth1Session
from requests.exceptions import ConnectionError, ReadTimeout, SSLError
import json, datetime, time, pytz, re, sys,traceback, pymongo
from pymongo import MongoClient
from collections import defaultdict
import numpy as np
# 【GitHub】に載せたくない環境変数の書き方 Python
#  https://qiita.com/hedgehoCrow/items/2fd56ebea463e7fc0f5b 
import os
from dotenv import load_dotenv


load_dotenv()

True

In [5]:
KEYS = { # 自分のアカウントで入手したキーを下記に記載
        'consumer_key':os.environ['TWITTER_API_KEY'],
        'consumer_secret':os.environ['TWITTER_API_SECRET_KEY'],
        'access_token':os.environ['TWITTER_ACCESS_TOKEN'],
        'access_secret':os.environ['TWITTER_ACCESS_TOKEN_SECRET'],
       }

twitter = None
connect = None
db      = None
tweetdata = None
meta    = None

def initialize(): # twitter接続情報や、mongoDBへの接続処理等initial処理実行
    global twitter, twitter
    twitter = OAuth1Session(KEYS['consumer_key'],KEYS['consumer_secret'],
                            KEYS['access_token'],KEYS['access_secret'])
initialize()


In [17]:
def serchTW(q):
    #ツイートを検索する用のエンドポイント
    url = 'https://api.twitter.com/1.1/search/tweets.json'
 
    params = {'q':q+" exclude:nativeretweets",
              'result_type':'mixed',
              "locale":"ja",
              'count':100}
 
    req = twitter.get(url, params = params)   # Tweetデータの取得
 
    serchtweet = json.loads(req.text) #json形式変換
    
    #取得したデータのツイート部分のみを表示
    for i in range(len(serchtweet["statuses"])):
        print("********************",i,"*************************")
        print(serchtweet["statuses"][i]["user"]["name"],":",serchtweet["statuses"][i]["text"])
    
#取得したツイートデータ全体を返す    
    return serchtweet["statuses"]

In [18]:
tweet=serchTW('夏祭り')

******************** 0 *************************
手越祐也 : Twitter夏祭りに参加します！皆から「手越祐也に挑戦して欲しいこと」を募集するよ！
8月31日までに@YuyaTegoshi1054 をフォローして　#Twitter夏祭り #手越屋台 をつけて投稿してくれた人の中から5名に俺… https://t.co/VFwz0tujcm
******************** 1 *************************
お買いものパンダ【楽天公式】 : おうちで夏祭り。わっしょーい！ #お買いものパンダ https://t.co/7t1ezvwFQ7
******************** 2 *************************
MINTIA（ミンティア） : 夏といえば夏祭り！
スマホでも気分を楽しめるように
射的ゲームをつくってみました🎯

動画をタップして、
特賞のミンティアを当てられた人はRT🔁

３回以内にできるかな？ https://t.co/OKTSbRb7Wz
******************** 3 *************************
Rain🍠@あめもん : おはようかなもん
今日はおはツイにしてみたよ
昨日言ってた浴衣着るなら🍧食べたりしてかなもんの夏祭り配信とか面白そうじゃない？😁
かなもん浴衣似合うだろうなぁー☺️
あ、でも地雷メイクも見てみたいかもどんなふうになるのか気になるな… https://t.co/F7YQgfWfBh
******************** 4 *************************
🍵💘渚🕊🍇 : @m__jel 仲間やん！笑
自撮り待ってます🥺w

楽しみwww

夏祭りも花火大会も今年は無いもんね😢

今日もよろしくね！
******************** 5 *************************
夢蔵(MUSASHI)@和楽器教材YouTube配信中！ : 和太鼓といったらこれ！【盆太鼓】ですね✨

こちらは盆太鼓講座動画です！
夏には盆祭りで太鼓デビューを♬
https://t.co/qAUUvqY9dY
******************** 6 *******

In [20]:
# # データベースを削除(再度やり直したい時)
# db.drop_collection('tweet')

In [21]:
# MongoDBに接続
client = pymongo.MongoClient('localhost', 27017)
 
# データベース「test_db」を作成
db = client.test_db
 
# tweetというコレクション(RDBのテーブルと同義）を作成
db_tweet = db.tweet2

# コレクションに取得したtweetデータをインサート
db_tweet.insert_many(tweet)

In [22]:
list(db_tweet.find())

s': False,
  'retweet_count': 0,
  'favorite_count': 0,
  'favorited': False,
  'retweeted': False,
  'lang': 'ja'},
 {'_id': ObjectId('5f38869207b9d1b40489d884'),
  'created_at': 'Sun Aug 16 00:47:21 +0000 2020',
  'id': 1294797861754818561,
  'id_str': '1294797861754818561',
  'text': '@555HOLiC あ～～そのパターンでダブると絶望ですよね…😭😭\n\nufoカフェは一応自分で選べるんですけど、今回の夏祭り缶バッジFFさんも現地でお話しした方も大量に買ったのに同じキャラ3～4種しか出なくて、そもそも全種混ぜてな… https://t.co/5IyiCCcMbS',
  'truncated': True,
  'entities': {'hashtags': [],
   'symbols': [],
   'user_mentions': [{'screen_name': '555HOLiC',
     'name': 'ホリック',
     'id': 536205051,
     'id_str': '536205051',
     'indices': [0, 9]}],
   'urls': [{'url': 'https://t.co/5IyiCCcMbS',
     'expanded_url': 'https://twitter.com/i/web/status/1294797861754818561',
     'display_url': 'twitter.com/i/web/status/1…',
     'indices': [117, 140]}]},
  'metadata': {'iso_language_code': 'ja', 'result_type': 'recent'},
  'source': '<a href="http://twitter.com/download/android" rel="nofollow">Tw